In [23]:
import sys
print(sys.version)

3.7.12 (default, Oct 13 2021, 06:53:03) 
[Clang 13.0.0 (clang-1300.0.29.3)]


In [24]:
%load_ext autoreload
%autoreload 0

import os, json
from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from scipy import stats
from spread import get_metabet_spread, get_current_rankings, get_fantasy_def_points_against
from collections import defaultdict

# https://www.pro-football-reference.com/years/2021/fantasy.htm

SALARY_FILE = './data/1_17.csv'
WEEK = 20
MIN_SALARY = 4799
WEIGHTED = True

ACTIVE_FILE = './data/active.csv'
MAX_PLAYED = WEEK - 1
print('ready')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready


In [25]:
# https://www.lineups.com/nfl-team-rankings
ranking_df = get_current_rankings(WEEK, 'MTI0NTQ2MDU3OQ==', True)

def get_abbr(x):
    try:
        if type(x) is not dict:
            x = eval(x)
        return x['team_abbr']
    except Exception as e:
        print(e,x)
        

print(ranking_df.shape)
ranking_df['team'] = ranking_df['nav'].apply(get_abbr)
# DEF_KEYS = ['passing_interceptions_rank'
# ranking_df['def_rank'] = ranking_df.apply(lambda row: np.avg([row[k] for k in DEF_KEYS]))
num_teams = ranking_df.shape[0]


return cached data week 19
(32, 52)


In [26]:
rankings = {x['team']: x for x in ranking_df.to_dict('records')}
# rankings
teams = set(rankings.keys())
print(teams, len(teams))
rankings['LV'] = rankings['OAK']
rankings['JAC'] = rankings['JAX']
del rankings['OAK']
rankings['PHI']

{'JAX', 'IND', 'ATL', 'OAK', 'LAC', 'CAR', 'NYJ', 'DEN', 'PHI', 'MIA', 'NO', 'BUF', 'CHI', 'SEA', 'MIN', 'GB', 'HOU', 'NE', 'WAS', 'TEN', 'CLE', 'CIN', 'PIT', 'NYG', 'ARI', 'LAR', 'BAL', 'TB', 'DET', 'SF', 'DAL', 'KC'} 32


{'Unnamed: 0': 2,
 'red_zone_conversions_rank': 27,
 'offensive_plays': 1076,
 'fantasy_points': 107,
 'third_down_percentage': 42.9,
 'fourth_down_percentage_rank': 11,
 'offensive_plays_rank': 18,
 'points_rank': 2,
 'third_down_percentage_rank': 23,
 'sacks': 29,
 'team_fk__full_name': 'Philadelphia Eagles',
 'sacks_rank': 31,
 'turnover_rank': 26,
 'passing_attempts_rank': 16,
 'points_rank_def': 18,
 'red_zone_attempts_rank': 23,
 'passing_yards_rank': 11,
 'red_zone_percentage': 66.7,
 'passing_interceptions': 12,
 'offensive_rating_rank': 3,
 'first_downs': 358,
 'points_def': 385,
 'red_zone_attempts': 60,
 'rushing_touchdowns': 18,
 'offensive_yards_rank': 10,
 'fourth_down_percentage': 48.3,
 'first_downs_rank': 23,
 'passing_completions': 409,
 'offensive_yards': 5590,
 'red_zone_conversions': 40,
 'rushing_yards': 1834,
 'passing_touchdowns': 28,
 'fantasy_points_rank': 16,
 'passing_touchdowns_rank': 20,
 'touchdowns': 47,
 'points': 30,
 'rushing_attempts_rank': 18,
 'rus

In [27]:

spread_df = get_metabet_spread(WEEK)
favor_map = {}
z_map = {}
z_scores = {}

if 'OverUnder' in spread_df.columns.values:
    points = list(spread_df['OverUnder'])
    zs = stats.zscore(points)
    for i, p in enumerate(points):
        z_scores[p] = zs[i]

# https://madduxsports.com/how-to-read-nfl-odds-lines.html"
for index, row in spread_df.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    favor_map[home] = row['PointSpread']
    favor_map[away] = -row['PointSpread']
    if 'OverUnder' in row:
        z_map[home] = z_scores[row['OverUnder']]
        z_map[away] = z_scores[row['OverUnder']]
    

    
if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    z_map['JAC'] = z_map['JAX']
    
if 'LVS' in favor_map:
    favor_map['LV'] = favor_map['LVS']
    z_map['LV'] = z_map['LVS']

print('favor_map teams', len(favor_map))
print(favor_map)
z_map

return cached data week 19
favor_map teams 13
{'CIN': -5.77, 'LVS': 5.77, 'BUF': -4.55, 'NE': 4.55, 'TB': -8.38, 'PHI': 8.38, 'DAL': -3.02, 'SF': 3.02, 'KC': -12.48, 'PIT': 12.48, 'LAR': -3.95, 'ARI': 3.95, 'LV': 5.77}


{'CIN': 0.4515007627679863,
 'LVS': 0.4515007627679863,
 'BUF': -1.427458204239028,
 'NE': -1.427458204239028,
 'TB': -0.7667253806761238,
 'PHI': -0.7667253806761238,
 'DAL': 1.4425999981123454,
 'SF': 1.4425999981123454,
 'KC': -0.5726351137545176,
 'PIT': -0.5726351137545176,
 'LAR': 0.8727179377893405,
 'ARI': 0.8727179377893405,
 'LV': 0.4515007627679863}

In [28]:
df = pd.read_csv(SALARY_FILE, na_values= '')
df = df.fillna(df.median())
print(df.describe())

df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']

all_teams = favor_map.keys()
winning_teams = [x for x in all_teams if favor_map[x] < 0]
ordered_teams = sorted(all_teams, key=lambda x: favor_map[x])
print('ordered_teams', [(x, favor_map[x]) for x in ordered_teams])
# excluded_teams = set([x for x in all_teams if favor_map[x] > -MIN_FAVORED])
# top_teams = winning_teams
team_offset = 1
half_teams = int(len(ordered_teams)/2)
top_teams = ordered_teams#[team_offset:team_offset+int(half_teams)]
print(f"top_teams {top_teams}")
excluded_teams = set(all_teams) - set(top_teams)
print(f"excluded_teams {excluded_teams}")

questionable_players = list(df[(~df['Injury Indicator'].isna()) | (~df['Injury Details'].isna())]['Name'])
low_salary_players = list(df[((df['Salary'] < MIN_SALARY)) & (df['Position'] != 'D')]['Name'])

excluded_players = [*questionable_players, *low_salary_players]
excluded_teams = list(df[(df['Position'] == 'D') & df['Team'].isin(excluded_teams)]['Name'])

if excluded_teams:
    excluded_players.extend(excluded_teams)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {len(excluded_players)}")

# custom_adds = ['Mike Evans', 'Tee Higgins', 'Los Angeles Rams', 'Tyler Higbee', 'Gabriel Davis', 'Sony Michel', 'Mecole Hardman']
# custom_adds = ['Byron Pringle', 'Tee Higgins', 'Gabriel Davis', 'Cedrick Wilson', 'Tyler Boyd', 'Tony Pollard','Mecole Hardman', 'Ezekiel Elliott', 'Tyler Higbee']
# custom_adds = ['Tyler Boyd', 'Stefon Diggs', 'Tyler Higbee']
# custom_adds = ['AJ Dillon', 'Tyler Boyd']
custom_adds = []

if excluded_players:
    excluded_players.extend(custom_adds)
    
questionable_df = df[(df['Name'].isin(set(questionable_players)))]
    
excluded_df = df[(df['Name'].isin(set(excluded_players)))]
    
df = df[(~df['Name'].isin(set(excluded_players)))]# & (df['FPPG'] > 0) & (df['Played'] > 0)

             FPPG      Played        Salary  Tier  Unnamed: 14  Unnamed: 15
count  379.000000  379.000000    379.000000   0.0          0.0          0.0
mean     4.622991    7.828496   5014.511873   NaN          NaN          NaN
std      5.010672    5.467521   1092.337652   NaN          NaN          NaN
min     -0.666667    1.000000   3100.000000   NaN          NaN          NaN
25%      0.975000    3.000000   4500.000000   NaN          NaN          NaN
50%      2.938889    7.000000   4500.000000   NaN          NaN          NaN
75%      6.582721   14.000000   5450.000000   NaN          NaN          NaN
max     24.563529   17.000000  10000.000000   NaN          NaN          NaN
ordered_teams [('KC', -12.48), ('TB', -8.38), ('CIN', -5.77), ('BUF', -4.55), ('LAR', -3.95), ('DAL', -3.02), ('SF', 3.02), ('ARI', 3.95), ('NE', 4.55), ('LVS', 5.77), ('LV', 5.77), ('PHI', 8.38), ('PIT', 12.48)]
top_teams ['KC', 'TB', 'CIN', 'BUF', 'LAR', 'DAL', 'SF', 'ARI', 'NE', 'LVS', 'LV', 'PHI', 'PIT']
exclud

In [29]:
print(winning_teams)

['CIN', 'BUF', 'TB', 'DAL', 'KC', 'LAR']


In [30]:
# http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd&scsv=1

start_week = WEEK-5# take last few games for momentum weighting.

file_names = [f"./history/week{week_number}.csv" for week_number in range(start_week, WEEK+1) if os.path.isfile(f"./history/week{week_number}.csv")]
print(file_names, len(file_names))

history_dfs = [pd.read_csv(f, delimiter=";") for f in file_names]
print(f"Using {len(history_dfs)} weeks of history")
historic_data=pd.concat(history_dfs)

if len(historic_data) > 0:
    REPLACE_MAP = {
        'LA': 'Los Angeles',
        '.':'',
    }

    def name_map(x):
        result = ' '.join(x.split(', ')[::-1])
        for k in REPLACE_MAP:
            result = result.replace(k, REPLACE_MAP[k])
        return result


    historic_data['Name'] = historic_data['Name'].apply(name_map)
    team_data = historic_data[historic_data['Pos'] == 'Def']

    historic_data[:1]
    historic_averages = historic_data.groupby("Name").mean()['FD points'].to_dict()
    historic_averages['Patrick Mahomes'] = historic_averages['Patrick Mahomes II']
    historic_averages['Darrell Henderson Jr'] = historic_averages['Darrell Henderson']
    # historic_averages

    team_averages = team_data.groupby("Team").mean()['FD points'].to_dict()
    team_averages['gb'] = team_averages.get('gnb')
    team_averages['kc'] = team_averages.get('kan')
    team_averages['ne'] = team_averages.get('nwe')
    team_averages['tb'] = team_averages.get('tam')
    team_averages['lv'] = team_averages.get('lvr')
    team_averages['no'] = team_averages.get('nor')
    team_averages['sf'] = team_averages.get('sfo')
    # print(team_averages)
    
    for k,v in team_averages.items():
        historic_averages[k] = team_averages[k]
print(len(historic_averages))

['./history/week14.csv', './history/week15.csv', './history/week16.csv', './history/week17.csv', './history/week18.csv'] 5
Using 5 weeks of history
661


In [31]:
excluded_bonus = defaultdict(lambda: 0)

for index, p in questionable_df.iterrows():
    pos = p['Position']
    if pos in ['TE', 'WR', 'RB', 'QB']:
        points = p['FPPG']
        if points > 7:
            if pos == 'QB':
                # subtract for QB
                amt = -min(points * .08, .8)
            else:
                amt = min(points * .05, .6)
            print('bonus', p['Team'], p['Name'], amt)
            
            excluded_bonus[p['Team']] += amt
        
print(excluded_bonus)

bonus ARI James Conner 0.6
bonus PIT Najee Harris 0.6
bonus NE Damien Harris 0.6
bonus TB Leonard Fournette 0.6
bonus KC Darrel Williams 0.5073529411764706
bonus KC Clyde Edwards-Helaire 0.5404999923706055
bonus ARI Chase Edmonds 0.5075000127156576
bonus LAR Darrell Henderson Jr. 0.6
bonus NE Jakobi Meyers 0.4525000095367432
bonus LAR Van Jefferson 0.42117646161247707
bonus TB Cyril Grayson 0.6
bonus ARI DeAndre Hopkins 0.6
bonus NE James White 0.42000001271565757
bonus LV Kenyan Drake 0.3645833333333334
bonus DAL Michael Gallup 0.4111111111111111
bonus TB Chris Godwin 0.6
bonus TB Antonio Brown 0.6
bonus LAR Robert Woods 0.6
defaultdict(<function <lambda> at 0x152f56c20>, {'ARI': 1.7075000127156574, 'PIT': 0.6, 'NE': 1.4725000222524007, 'TB': 2.4, 'KC': 1.047852933547076, 'LAR': 1.6211764616124769, 'LV': 0.3645833333333334, 'DAL': 0.4111111111111111})


In [32]:
df.to_csv(ACTIVE_FILE)
print(f"wrote {ACTIVE_FILE}, {df.shape}")

wrote ./data/active.csv, (131, 19)


In [33]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2 ,3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]               

In [34]:
m_score = df.groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
    
m_score

{'D': 7.509803921568628,
 'QB': 7.836579158762899,
 'RB': 7.466331082719559,
 'TE': 9.518029180401586,
 'WR': 8.412382179783194,
 'DEF': 7.509803921568628}

In [35]:
max_salary = np.percentile(df[df['Salary'] >= MIN_SALARY]['Salary'], 99)-1
max_salary = 9500
print(max_salary)

9500


In [36]:
df[df['Position'] == 'D'].shape

(12, 19)

In [37]:
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.salary_min = ACTIVE_RULE_SET.salary_max - 0
ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 60000, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}


In [38]:
# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
# Use salary data from csv as optimization basis.
# set((k, v['defensive_rating_rank']) for k,v in rankings.items())
# rankings['BUF']

allowed_map = get_fantasy_def_points_against(WEEK)
allowed_map

fetching data


{'Jacksonville Jaguars': {'rank': 32, 'allowed': 9.71},
 'Atlanta Falcons': {'rank': 31, 'allowed': 9.59},
 'New York Giants': {'rank': 30, 'allowed': 9.59},
 'Carolina Panthers': {'rank': 29, 'allowed': 9.47},
 'Chicago Bears': {'rank': 28, 'allowed': 9.12},
 'Baltimore Ravens': {'rank': 27, 'allowed': 8.35},
 'New York Jets': {'rank': 26, 'allowed': 8.24},
 'Houston Texans': {'rank': 25, 'allowed': 7.65},
 'Washington Football Team': {'rank': 24, 'allowed': 7.53},
 'Denver Broncos': {'rank': 23, 'allowed': 7.53},
 'Miami Dolphins': {'rank': 22, 'allowed': 7.41},
 'Las Vegas Raiders': {'rank': 21, 'allowed': 6.94},
 'Detroit Lions': {'rank': 20, 'allowed': 6.65},
 'Cleveland Browns': {'rank': 19, 'allowed': 6.53},
 'New Orleans Saints': {'rank': 18, 'allowed': 6.47},
 'Tennessee Titans': {'rank': 17, 'allowed': 6.41},
 'Cincinnati Bengals': {'rank': 16, 'allowed': 6.24},
 'Pittsburgh Steelers': {'rank': 15, 'allowed': 6.06},
 'New England Patriots': {'rank': 14, 'allowed': 5.71},
 'Lo

In [55]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)

FAVOR_DIVISION = 10
AVERAGE_WEIGHT = .6
OVERUNDER_DIVISION = 4

defenses = []
qbs = []

historic_data_used = 0

for p in players:
    p.average_score = m_score[p.pos]
    name = p.name.replace('.', '')
    
    if WEIGHTED:
        base_score = p.proj
        
        history_key = name_map(p.name) if p.pos != 'D' else p.team.lower()
        history_value = historic_averages.get(history_key)
        if history_value:
            new_score = AVERAGE_WEIGHT*base_score + (1-AVERAGE_WEIGHT)*history_value
            base_score = new_score
            historic_data_used+=1
        else:
            new_score = None
            # print('no historic data', history_key)
            
        
        rank_bonus = -favor_map.get(p.team, 0)/FAVOR_DIVISION
        
        # print('rank_bonus', rank_bonus)
        teams = p.matchup.split('@')
        
        
        is_home = p.team == teams[1]
        if is_home:
            rank_bonus += 0.20

        teams.remove(p.team) # remove current players team
        opponent = teams[0]
        
        point_bonus = z_map.get(p.team, 0)
        if point_bonus:
            # max(0, point_bonus/4)
            overunder_bonus = -point_bonus/OVERUNDER_DIVISION if p.pos == 'D' else point_bonus/OVERUNDER_DIVISION # lower points better for defensive scoring.
            rank_bonus += overunder_bonus
            
        # current_rank = rankings[p.team]['overall_rating_rank']
        # opp_rank = rankings[opponent]['overall_rating_rank']
        current_rank = rankings[p.team]['points_rank']
        opp_rank = num_teams - rankings[opponent]['offensive_rating_rank']

        if p.pos == 'D':
            opp_bonus = excluded_bonus[opponent]/2
            rank_bonus += opp_bonus
            opp_def_avg = allowed_map[rankings[opponent]['team_fk__full_name']]['allowed']
            base_score = .5*base_score + .5*opp_def_avg
        else:
            if p.pos == 'QB':
                # negative (downside for QB injuries)
                injury_bonus = -excluded_bonus[p.team]
            else:
                injury_bonus = excluded_bonus[p.team] 
                
            rank_bonus += injury_bonus
            
        # overall_diff = opp_rank - current_rank # larger the better (should be between 1-32)
        # rank_bonus += overall_diff/num_teams*1.5
        p.proj = base_score + rank_bonus
        
        if p.pos == 'D':
            defenses.append((p.team, current_rank, opp_def_avg, opp_bonus, rank_bonus, base_score, p.proj, p.cost, p.proj / p.cost))
        elif p.pos == 'QB':
            qbs.append((history_key, history_value, base_score, overunder_bonus, p.proj, p.cost, p.proj / p.cost))
        
        if 'Jenn' in name or 'Uzomah' in name:# or p.pos == 'TE':
            print(history_key, history_value, base_score, overunder_bonus, p.proj, p.cost, p.proj / p.cost)
        

print(f"players {len(players)}")
print(f"historic data used {historic_data_used} of {len(players)}") 
# print(players)

for x in sorted(defenses, key=lambda x: x[-1], reverse=True):
    print(x)
# print(excluded_bonus)
print("\n---\n")
for x in sorted(qbs, key=lambda x: x[-1], reverse=True):
    print(x)

Jauan Jennings 9.44 7.015999859149639 0.36064999952808635 7.074649858677725 5200.0 0.0013605095882072548
CJ Uzomah 5.55 6.112500114440918 0.11287519069199657 7.002375305132914 5200.0 0.0013466106356024834
players 131
historic data used 107 of 131
('DAL', 1, 5.18, 0.0, 0.14135000047191365, 8.57470588235294, 8.716055882824854, 4100.0, 0.002125867288493867)
('BUF', 9, 5.71, 0.7362500111262004, 1.7481145621859575, 7.13264705882353, 8.880761621009487, 4500.0, 0.0019735025824465526)
('KC', 4, 6.06, 0.3, 1.8911587784386295, 7.381764705882354, 9.272923484320984, 4800.0, 0.0019318590592335384)
('TB', 9, 4.24, 0.0, 1.229681345169031, 6.088235294117648, 7.317916639286679, 4200.0, 0.0017423611045920663)
('CIN', 9, 6.94, 0.1822916666666667, 0.8464164759746701, 6.4558823529411775, 7.3022988289158475, 4300.0, 0.0016982090299804296)
('PIT', 30, 4.59, 0.523926466773538, -0.5809147547878325, 5.603823529411764, 5.022908774623932, 3100.0, 0.0016202931531044942)
('LAR', 30, 4.65, 0.8537500063578287, 1.2305

In [40]:
player_settings = PlayerPoolSettings()
LOCKED = []
BLOCKED_TEAMS = []

# LOCKED = ['Justin Herbert','Ronald Jones II','Dallas Goedert','Miami Dolphins']# 'Miami Dolphins']#['Justin Herbert',  'Tee Higgins', 'Ronald Jones II', 'David Montgomery', 'Miami Dolphins']
# LOCKED = [ 'Justin Herbert', 'Ronald Jones II','Dallas Goedert', 'Cincinnati Bengals', 'David Montgomery']
# LOCKED = ['Joe Burrow', 'Brandon Aiyuk']
# LOCKED = ['Justin Jackson', 'Darrel Williams', 'Dak Prescott', 'Dalton Schultz']
# LOCKED = ['Darrel Williams']
# LOCKED = ['Joe Burrow']
# LOCKED = ['Tee Higgins']
# LOCKED = ['Dallas Goedert', 'Byron Pringle']
# LOCKED = ['David Montgomery', 'Byron Pringle', 'Dallas Goedert']
# LOCKED = ['Dallas Goedert']#, 'Dak Prescott']
# LOCKED = ['Devin Singletary', 'Jaylen Waddle', 'Dallas Goedert', 'Tee Higgins']
# LOCKED = ['Dak Prescott', 'Joe Mixon', 'Tee Higgins']#, 'Devin Singletary', 'Tee Higgins', 'Dallas Goedert']
# LOCKED = ['Aaron Jones','Rhamondre Stevenson']#, 'Nick Chubb', 'Devin Singletary']#, 'Amon-Ra St. Brown']#,  'Dallas Goedert']
# LOCKED = ['Aaron Jones',  'Zach Ertz', 'Devin Singletary']#,'Rhamondre Stevenson', 'Devin Singletary', 'Zach Ertz']
# LOCKED = ['Buffalo Bills', 'Zach Ertz', 'AJ Dillon', 'Rhamondre Stevenson']#, 'Cam Akers'] # 'Devin Singletary',  'Rhamondre Stevenson', 
# LOCKED = ['AJ Dillon', 'Devin Singletary', 'Rhamondre Stevenson', 'Buffalo Bills', 'Stefon Diggs', 'Cyril Grayson']
# LOCKED = ['Buffalo Bills']#, 'Devin Singletary']
# LOCKED = ['Tampa Bay Buccaneers']#, 
# LOCKED = ['Hunter Renfrow']
# LOCKED = [ 'Mike Evans', 'Deebo Samuel', 'Tee Higgins', 'Devin Singletary']#, 'Cedrick Wilson']#, 'Mike Evans']
# LOCKED = ['Dallas Cowboys'] 'Philadelphia Eagles', 
# LOCKED = ['Pittsburgh Steelers', 'Mike Evans']
# LOCKED = ['Joe Mixon', 'Zay Jones', 'Dallas Cowboys']
# LOCKED = ['Zay Jones', 'Tee Higgins', 'Rhamondre Stevenson']#, 'Rhamondre Stevenson']#, 'Joe Mixon', 'Deebo Samuel', 'Tee Higgins', 'Devin Singletary']
# LOCKED = ['Rhamondre Stevenson']
# LOCKED = ['Rhamondre Stevenson', 'Devin Singletary', 'Joe Mixon', 'Tee Higgins', 'Joe Burrow', 'Dallas Cowboys']#, "Joe Mixon"]
# LOCKED = ['Tee Higgins', 'Zay Jones', 'Joe Mixon', 'Rhamondre Stevenson']
# LOCKED = ['Cincinnati Bengals', 'Davante Adams']
 
# BLOCKED_TEAMS = ['TEN', 'WAS']#['WAS', 'TEN']#['BUF']#['SEA', 'LV']

constraints = LineupConstraints(locked=LOCKED)

MIN_PROJ = 5
min_favored = -8
MIN_LIMIT = -10

roster = None
best_roster = None
best_score = 0

get_score = lambda roster: sum([p.proj for p in roster.players])

while min_favored >= MIN_LIMIT:
    def block_function(p):
        store = p.kv_store
        played = int(float(store.get('Played', -1)))
        
        should_skip = ((p.team in BLOCKED_TEAMS or p.proj < MIN_PROJ) and p.pos != 'D') or (p.cost > max_salary) or played < MAX_PLAYED/2 or (p.pos == 'QB' and p.proj < 12)# or (favor_map.get(p.team, 0) > -min_favored)
    
        #print(p.name, played, MAX_PLAYED)
        return should_skip

    opt_settings = OptimizerSettings(
        custom_rules=[
            CustomRule(
                group_a=lambda p: p,
                group_b=block_function, 
                comparison=lambda sum, a, b: sum(b) == 0
            ),
        ],
        min_teams=2
    )

    roster = run(
        rule_set=ACTIVE_RULE_SET,
        player_pool=players,
        verbose=False,
        optimizer_settings=opt_settings,
        constraints=constraints,
        player_settings=player_settings
    )
    
    if roster:
        # total_salary = sum([p.cost for p in roster.players])
        print(roster)
        print('min_favored', min_favored)# negative means players' teams can lose
        print(f"spread weighted: {WEIGHTED}\n\n")
        current_score = get_score(roster)
        if not best_score or current_score > best_score:
            best_score = current_score
            best_roster = roster

    min_favored -= 1
    
    if min_favored >= MIN_LIMIT:
        roster = None
        
if not roster:
    print("No solution")
elif not favor_map:
    print("Warning: No favor map used")

+----------+---------------------+------+---------+--------+--------------------+----------+--------+
| Position | Player              | Team | Matchup | Salary |         Projection | vs. Avg. | Locked |
+----------+---------------------+------+---------+--------+--------------------+----------+--------+
| QB       | Joe Burrow          | CIN  | LV@CIN  |  7,800 |  23.99287482448106 |    16.16 |        |
| RB       | Tony Pollard        | DAL  | SF@DAL  |  5,700 | 10.131094688113157 |     2.66 |        |
| RB       | Rhamondre Stevenson | NE   | NE@BUF  |  5,200 | 10.228968880819922 |     2.76 |        |
| WR       | Deebo Samuel        | SF   | SF@DAL  |  8,700 | 18.970699679093517 |    10.56 |        |
| WR       | Ja'Marr Chase       | CIN  | LV@CIN  |  8,200 | 17.675051876698433 |     9.26 |        |
| WR       | Tee Higgins         | CIN  | LV@CIN  |  6,900 | 14.924161166556955 |     6.51 |        |
| TE       | Travis Kelce        | KC   | PIT@KC  |  7,500 | 16.742694269549368 | 

In [41]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:||
#         return matches.iloc[0]['Id']
#     return None
#print(f"{"Player":20}{"Advantage":10}")
if favor_map:
    roster = best_roster
    sorted_players = sorted(roster.players, key=lambda x: favor_map[x.team])
    net_score = 0
    for p in sorted_players:
        advantage = favor_map[p.team]
        name = p.name.replace('.', '')
        print(f"{name:24}{advantage:>10}{historic_averages.get(name, ''):>20} {p.proj} {p.kv_store['Played']} {excluded_bonus[p.team]}")#, p.kv_store['Played'], MAX_PLAYED)
        net_score += advantage
    print(f"---\nTotal adv: {net_score}\n")
    roster.players[-1].__dict__



Travis Kelce                -12.48               15.65 16.742694269549368 16.0 1.047852933547076
Ja'Marr Chase                -5.77               18.66 17.675051876698433 17.0 0
Joe Burrow                   -5.77              26.985 23.99287482448106 16.0 0
Tee Higgins                  -5.77              15.575 14.924161166556955 14.0 0
Tyler Higbee                 -3.95  12.333333333333334 12.027689279393146 15.0 1.6211764616124769
Tony Pollard                 -3.02   7.833333333333333 10.131094688113157 15.0 0.4111111111111111
Dallas Cowboys               -3.02                     8.716055882824854 17.0 0.4111111111111111
Deebo Samuel                  3.02  19.112000000000002 18.970699679093517 16.0 0
Rhamondre Stevenson           4.55  10.433333333333332 10.228968880819922 12.0 1.4725000222524007
---
Total adv: -32.21000000000001



In [42]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])
    # if 'Zay' in r.name:
    #     print(r.__dict__)

SF Deebo Samuel 18.970699679093517 8.412382179783194 8700.0   16.0
CIN Ja'Marr Chase 17.675051876698433 8.412382179783194 8200.0   17.0
CIN Joe Burrow 23.99287482448106 7.836579158762899 7800.0   16.0
KC Travis Kelce 16.742694269549368 9.518029180401586 7500.0   16.0
CIN Tee Higgins 14.924161166556955 8.412382179783194 6900.0   14.0
LAR Tyler Higbee 12.027689279393146 9.518029180401586 5900.0   15.0
DAL Tony Pollard 10.131094688113157 7.466331082719559 5700.0   15.0
NE Rhamondre Stevenson 10.228968880819922 7.466331082719559 5200.0   12.0
DAL Dallas Cowboys 8.716055882824854 7.509803921568628 4100.0   17.0


In [43]:
# Create upload CSV by reordering columns to match template order.

headers = []
players = []
ORDERED_COLS = ['QB','RB','RB','WR','WR','WR','TE','FLEX','DEF'] # template order.

def get_match_names(col):
    if col == 'DEF':
        return ['D']
    elif col == 'FLEX' :
        return ['RB', 'WR']
    return [col]

roster_copy = roster.players.copy()
for c in ORDERED_COLS:
    headers.append(c)
    match_names = get_match_names(c)
    for r in roster_copy:
        if r.pos in match_names:
            p = f"{r.kv_store['Id']}:{r.name}"
            players.append(p)
            roster_copy.remove(r)
            break

with open('upload.csv', 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


done


In [44]:
output = pd.read_csv('upload.csv')
output

,QB,RB,RB.1,WR,WR.1,WR.2,TE,FLEX,DEF
0,70039-63336:Joe Burrow,70039-63759:Tony Pollard,70039-111540:Rhamondre Stevenson,70039-56018:Deebo Samuel,70039-85701:Ja'Marr Chase,70039-89981:Tee Higgins,70039-33076:Travis Kelce,70039-12530:Dallas Cowboys,NaN


In [45]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score['FLEX'] = (m_score['WR']+m_score['RB'])/2
m_score

{'D': 7.509803921568628,
 'QB': 9.479698397051239,
 'RB': 7.466331082719559,
 'TE': 9.518029180401586,
 'WR': 8.412382179783194,
 'DEF': 7.509803921568628,
 'FLEX': 7.939356631251377}

In [46]:
expected_score = sum([m_score.get(h) for h in headers])
expected_score

74.61669683506153

In [47]:
# Players that have non-null injury status.
df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [48]:
df[df['First Name'] == 'Zay']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG
103,70039-41305,WR,Zay,Zay Jones,Jones,5.493333,15.0,5300,LV@CIN,LV,CIN,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Zay Jones,0.001036


In [49]:
# Potentially unaccounted positions
df[~df['Position'].isin(ALL_POSITIONS)]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [50]:
df[df['Position'] == 'D'][['FPPG', 'Nickname', 'Salary', 'Salary/FPPG']].sort_values('Salary/FPPG', ascending=False)

,FPPG,Nickname,Salary,Salary/FPPG
322,10.882353,Dallas Cowboys,4100,0.002654
374,9.294118,New England Patriots,3800,0.002446
378,6.529412,Pittsburgh Steelers,3100,0.002106
236,9.058824,Buffalo Bills,4500,0.002013
377,6.529412,Philadelphia Eagles,3300,0.001979
312,8.294118,Tampa Bay Buccaneers,4200,0.001975
375,7.000000,San Francisco 49ers,3700,0.001892
350,7.470588,Arizona Cardinals,4000,0.001868
132,7.705882,Kansas City Chiefs,4800,0.001605
138,7.117647,Los Angeles Rams,4700,0.001514


In [51]:
df[df['Team'] == 'JAC']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [52]:
spread_df.iloc[1][['HomeTeam', 'PointSpread']]

HomeTeam        BUF
PointSpread   -4.55
Name: 1, dtype: object

In [53]:
from pydfs_lineup_optimizer import Site, Sport, get_optimizer

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)
optimizer.load_players_from_csv(ACTIVE_FILE)
for lineup in optimizer.optimize(10):
    print(lineup)


 1. QB      Jalen Hurts                   QB    PHI            PHI@TB   21.411         7700.0$   
 2. RB      Jordan Howard                 RB    PHI            PHI@TB   8.786          5200.0$   
 3. RB      Rhamondre Stevenson           RB    NE             NE@BUF   8.992          5200.0$   
 4. WR      Cooper Kupp                   WR    LAR            ARI@LAR  21.588         10000.0$  
 5. WR      Deebo Samuel                  WR    SF             SF@DAL   18.779         8700.0$   
 6. WR      Tee Higgins                   WR    CIN            LV@CIN   13.007         6900.0$   
 7. TE      George Kittle                 TE    SF             SF@DAL   11.607         6500.0$   
 8. FLEX    Tony Pollard                  RB    DAL            SF@DAL   9.54           5700.0$   
 9. DEF     Dallas Cowboys                D     DAL            SF@DAL   10.882         4100.0$   

Fantasy Points 124.59
Salary 60000.00

 1. QB      Josh Allen                    QB    BUF            NE@BUF   24.564